# 1.2 shopbop crawling:
creating functiones to take: rating, size, fabric, color, name, pricre and id.
(the fabric fanction used selenium)

runing through all links of the website and creating a DF with the above, brand name and numerical order.

after finding a problom with the initial df, we redownloaded the problematic rows.

combaining the df to create a final df.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time 
import urllib.request
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

from selenium.webdriver.common.by import By

import re

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np

In [15]:
driverV = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\linoy\AppData\Local\Temp\ipykernel_41812\3481517888.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driverV = webdriver.Chrome(ChromeDriverManager().install())


In [16]:
def ratingV(driverV):
    
    soup = BeautifulSoup(driverV.page_source, 'html.parser')
    
    div=soup.find("div", {"class":"yotpoBottomLine"})
    if div==None: return None
    full_stars= div.find_all("span", {"class":"yotpo-icon yotpo-icon-star rating-star pull-left"})
    half_stars= div.find_all("span", {"class":"yotpo-icon yotpo-icon-half-star rating-star pull-left"})
    Rating= len(full_stars)+ 0.5*len(half_stars)
    return Rating

def sizeV(soup):
    div=soup.find("ul", {"class":"control-dropdown__values-list"})
    if div==None: return 'one_size'
    size_tags=div.find_all("span", {"class":"control-dropdown__size-num"})
    if size_tags==None: return 'one_size'
    size=[]
    for s in size_tags:
        size.append(s.string)
    return size


def fabricV(driverV):
    content=None
    
    try:
        driverV.find_element(By.XPATH,'//*[@id="tab-label-content.care-title"]')
    except NoSuchElementException:
            return content
    
    soup = BeautifulSoup(driverV.page_source, 'html.parser')

    for sec in soup.find_all("script",{"type":'application/ld+json'}): # עבור תכונות ספציפיות שתגזרו

        if sec==None: return None
        text=str(sec)#convert to string
        word_to_search = '"material"'
        index = text.find(word_to_search) # find specific elemnt -> check out where are all the others
        only_details = text[index+len(word_to_search):] # without the word "matirial"
        if(index!=-1): 
            content = re.findall('"([^"]*)"', only_details)[0] # prints only the "matiral's content, part between the quets
            break
            
    return content

def colorV(soup):
    c=soup.find("div", {"class":"color color--selected"})
    if(c!= None):
        color= c.attrs['title']
    else:
        color= None
    return color

def name_price_idV(soup):
    n=p=i=None
    
    name=soup.find("span", {"class":"base"})
    if name!=None: 
        n=name.string
    
    price=soup.find("meta", {"itemprop":"price"})
    if price!=None: 
        p=price.attrs['content']
    
    idp=soup.find("span", {"class":"sku"})
    if idp!=None: 
        i=idp.string
        
    return n, p, i

In [17]:
dead_page_list=[2555,2558,2418,2397,2398,2278,2214,2215,2216,2158,1468,638,639,260]

first try:

In [ ]:
dfV=pd.read_csv("avenue_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]

for num, v in enumerate(dfV["link V"]):
    driverV.get(v) 
    time.sleep(4)
    
    row = dict()
    row['num']=num
    soup = BeautifulSoup(driverV.page_source, 'html.parser')
    brands.append("avenue")
    row['brand']= "avenue"
    
    name, price, idp= name_price_idV(soup)

    product_names.append(name)
    row['product_names']= name

    prices.append(price)
    row['prices']= price

    ids.append(idp)
    row['ids']= idp
    
    rating=ratingV(soup)
    ratings.append(rating)
    row['ratings']= rating
    
    size=sizeV(soup)
    sizes.append(size)
    row['sizes']= size
    
    try:
        fabric=fabricV(driverV)
    exepct
    fabrics.append(fabric)
    row['fabrics']= fabric
    
    color=colorV(soup)
    colors.append(color)
    row['colors']= color
    
    temp_list.append(row)
    
    if num%100==0:
        df = pd.DataFrame(temp_list)
        df.to_csv("avenue_prodoct_info_final.csv", index=False)

In [2]:
df=pd.read_csv("avenue_prodoct_info_final.csv")
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,0,avenue,Ashley Pintuck Top - terracotta,34.50,Style # 217814TERRACOTTA,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","65% Viscose, 35% Polyester.",Terracotta
1,1,avenue,Elouise Maxi Dress - orange,39.50,Style # 217671SANDSOFTIME,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Viscose.,Sands Of Time
2,2,avenue,Elouise Top - orange,34.50,Style # 217670SANDSOFTIME,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Viscose.,Sands Of Time
3,3,avenue,Button Knit Cardigan - cream,49.50,Style # 217660CREAM,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","52% Acrylic, 26% Polyester, 22% Nylon.",Cream
4,4,avenue,Button Knit Cardigan - camel,49.50,Style # 217660CAMEL,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","52% Acrylic, 26% Polyester, 22% Nylon.",Camel
...,...,...,...,...,...,...,...,...,...
2621,2635,avenue,Butter Denim Skinny Jean Black - petite\n,NaN,Style # 203974BLACK,0.0,[None],"76% Cotton, 23% Polyester, 1% Spandex.",Black
2622,2636,avenue,Magnetic Pant - black,47.60,Style # 202574BLACK,0.0,[None],"Main: 97% Polyester, 3% Elastane",Black
2623,2637,avenue,Harley Hi Rise Skinny Jean - denim,19.99,Style # 201973DENIM,0.0,[None],"70% Cotton, 28% Polyester 2% Elastane",Denim
2624,2638,avenue,Asha Escape Skinny Jean - denim,19.99,Style # 203131LIGHTDENIM,0.0,[None],"77% Cotton, 21% Polyester, 2% Elastane",Light Denim


In [27]:
df.shape

(2626, 9)

there was a problom with some of the rows downloaded, especially at the rating column, therefore we redawnloaded those rows.

In [20]:
df_nan1=df.dropna()
df1= df_nan1[(df_nan1.ratings== 0.0)]

In [21]:
num_list=[]
for row in df1.num:
    num_list.append(row)
# num_list

In [69]:
dfV=pd.read_csv("avenue_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

row = dict()
temp_list=[]
errors_links = []

for num, v in enumerate(dfV["link V"]):
    
    if num in num_list:
        
        
        driverV.get(v) 
        time.sleep(5)
        soup = BeautifulSoup(driverV.page_source, 'html.parser')
        row = dict()
        row['num']=num

        brands.append("avenue")
        row['brand']= "avenue"

        name, price, idp= name_price_idV(soup)

        product_names.append(name)
        row['product_names']= name

        prices.append(price)
        row['prices']= price

        ids.append(idp)
        row['ids']= idp

        rating=ratingV(driverV)
        ratings.append(rating)
        row['ratings']= rating

        size=sizeV(soup)
        sizes.append(size)
        row['sizes']= size

        if (size==None):
            
            errors_links.append(num)
            print(num)


        fabric=fabricV(driverV)
        fabrics.append(fabric)
        row['fabrics']= fabric

        color=colorV(soup)
        colors.append(color)
        row['colors']= color

        temp_list.append(row)

        if num%10==0:
            df = pd.DataFrame(temp_list)
            df.to_csv("avenue_prodoct_info_temp_fix20.csv", index=False)

        
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,0,avenue,Ashley Pintuck Top - terracotta,34.5,Style # 217814TERRACOTTA,0.0,[None],"65% Viscose, 35% Polyester.",Terracotta
1,1,avenue,Elouise Maxi Dress - orange,39.5,Style # 217671SANDSOFTIME,0.0,"[14, 16, 18, 20, 22-24, 26-28, 30-32]",100% Viscose.,Sands Of Time
2,2,avenue,Elouise Top - orange,34.5,Style # 217670SANDSOFTIME,0.0,"[14, 16, 18, 20, 22-24, 26-28, 30-32]",100% Viscose.,Sands Of Time
3,3,avenue,Button Knit Cardigan - cream,49.5,Style # 217660CREAM,0.0,"[14, 16, 18, 20, 22-24, 26-28, 30-32]","52% Acrylic, 26% Polyester, 22% Nylon.",Cream
4,4,avenue,Button Knit Cardigan - camel,49.5,Style # 217660CAMEL,0.0,"[14, 16, 18, 20, 22-24, 26-28, 30-32]","52% Acrylic, 26% Polyester, 22% Nylon.",Camel
...,...,...,...,...,...,...,...,...,...
1972,2626,avenue,High Rise Jegging Dark Wash - average,27.6,Style # 206570DARKWASH,4.0,"[14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 3...","68% Cotton, 31% Polyester, 1% Elastane.",Dark Wash
1973,2627,avenue,High Rise Jegging Mid Wash - tall,19.99,Style # 206584MIDWASH,4.0,"[14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 3...","68% Cotton, 31% Polyester, 1% Elastane.",Mid Wash
1974,2628,avenue,High Rise Jegging Mid Wash - average,9.99,Style # 206582MIDWASH,4.0,"[14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 3...","68% Cotton, 31% Polyester, 1% Elastane.",Mid Wash
1975,2629,avenue,Curvy Tummy Control Trouser Black - petite,27.6,Style # 203920BLACK,4.0,"[14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 3...","72% Polyester, 24% Viscose, 4% Elastane",Black


In [10]:
dfV=pd.read_csv("avenue_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]
errors_links = []

for num, v in enumerate(dfV["link V"]):
    
    if num>2630:
    
        if num in num_list:


            driverV.get(v) 
            time.sleep(5)
            soup = BeautifulSoup(driverV.page_source, 'html.parser')
            row = dict()
            row['num']=num

            brands.append("avenue")
            row['brand']= "avenue"

            name, price, idp= name_price_idV(soup)

            product_names.append(name)
            row['product_names']= name

            prices.append(price)
            row['prices']= price

            ids.append(idp)
            row['ids']= idp

            rating=ratingV(driverV)
            ratings.append(rating)
            row['ratings']= rating

            size=sizeV(soup)
            sizes.append(size)
            row['sizes']= size

            if (size==None):

                errors_links.append(num)
                print(num)


            fabric=fabricV(driverV)
            fabrics.append(fabric)
            row['fabrics']= fabric

            color=colorV(soup)
            colors.append(color)
            row['colors']= color

            temp_list.append(row)

            if num%1==0:
                df = pd.DataFrame(temp_list)
                df.to_csv("avenue_prodoct_info_temp_fix25.csv", index=False)

        
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,2632,avenue,Stretch Corduroy Jean in Black - petite,27.6,Style # 214076462BLACKPETITE,3.5,"[14, 16, 18, 20, 22, 24, 26, 28, 30, 32]","78% Cotton, 20% Model, 2% Spandex. Fabric has ...",Black
1,2634,avenue,Cool Hand Pant Black - petite,27.6,Style # 203914BLACK,3.5,"[14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 3...","72% Polyester, 24% Viscose, 4% Elastane. Fabri...",Black
2,2636,avenue,Magnetic Pant - black,47.6,Style # 202574BLACK,4.0,"[12, 14, 16, 18, 20, 22, 24]","Main: 97% Polyester, 3% Elastane",Black
3,2637,avenue,Harley Hi Rise Skinny Jean - denim,19.99,Style # 201973DENIM,4.5,"[12, 14, 16, 18, 20, 22, 24]","70% Cotton, 28% Polyester 2% Elastane",Denim
4,2638,avenue,Asha Escape Skinny Jean - denim,19.99,Style # 203131LIGHTDENIM,4.0,"[12, 14, 16, 18, 20, 22, 24]","77% Cotton, 21% Polyester, 2% Elastane",Light Denim
5,2639,avenue,Harley So Jaded Skinny Jean - light wash,19.99,Style # 205504LIGHTWASH,3.0,"[12, 14, 16, 18, 20, 22, 24]","70% Cotton, 28% Polyester, 2% Elastane",Light Wash


running again over the ranked ones to complete the table (all of the ones we didnt run over at the previouse run)

In [12]:
dfV=pd.read_csv("avenue_link_test3.csv")
count=0
for num, v in enumerate(dfV["link V"]):
    
    if num not in dead_page_list and num not in num_list:
        count=count+1
        
count

643

In [ ]:
dfV=pd.read_csv("avenue_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]
errors_links = []

for num, v in enumerate(dfV["link V"]):
    
    if num not in dead_page_list and num not in num_list:

        driverV.get(v) 
        time.sleep(5)
        soup = BeautifulSoup(driverV.page_source, 'html.parser')
        row = dict()
        row['num']=num

        brands.append("avenue")
        row['brand']= "avenue"

        name, price, idp= name_price_idV(soup)

        product_names.append(name)
        row['product_names']= name

        prices.append(price)
        row['prices']= price

        ids.append(idp)
        row['ids']= idp

        rating=ratingV(driverV)
        ratings.append(rating)
        row['ratings']= rating

        size=sizeV(soup)
        sizes.append(size)
        row['sizes']= size

        if (size==None):

            errors_links.append(num)
            print(num)


        fabric=fabricV(driverV)
        fabrics.append(fabric)
        row['fabrics']= fabric

        color=colorV(soup)
        colors.append(color)
        row['colors']= color

        temp_list.append(row)

        if num%1==0:
            df = pd.DataFrame(temp_list)
            df.to_csv("avenue_prodoct_info_temp_fix30.csv", index=False)


df

In [23]:
df=pd.read_csv("avenue_prodoct_info_temp_fix30.csv")
df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,6,avenue,Rosie Cable Knit Sweater - cream,44.50,Style # 217658CREAM,3.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","60% Cotton, 40% Acrylic.",Cream
1,9,avenue,Leila Split Tunic - amethyst,34.50,Style # 215135AMETHYST,5.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","Main: 65% Viscose, 35% Polyester. Lining: 60% ...",Amethyst
2,10,avenue,Maria Print Tunic - amethyst,34.50,Style # 215129AMETHYSTGEM,5.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","95% Viscose, 5% Elastane.",Amethyst Gem
3,11,avenue,Karina Knit Dress - olive,34.50,Style # 215127OLIVE,4.5,"['14', '16', '18', '20', '22-24', '26-28', '30...","95% Viscose, 5% Elastane.",Olive
4,12,avenue,Karina Knit Dress - navy,34.50,Style # 215127NAVY,5.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","95% Viscose, 5% Elastane.",Navy
...,...,...,...,...,...,...,...,...,...
99,353,avenue,Polar Fleece Print Tunic - plum tie dye,14.99,Style # 218027PLUMTIEDYE,4.5,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Polyester.,Plum Tie Dye
100,354,avenue,Chloe Zip Plain Top - plum,12.99,Style # 217718PLUM,5.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","95% Polyester, 5% Elastane.",Plum
101,355,avenue,Frankie Frill Tunic - violet,31.60,Style # 217844VIOLET,3.0,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Viscose.,Violet
102,356,avenue,Rosey Crush Tunic - plum,12.99,Style # 217943PLUM,5.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","Main Body: 65% Viscose, 35% Polyester. Lace: 7...",Plum


In [24]:
dfV=pd.read_csv("avenue_link_test3.csv")

ratings=[]
prices=[]
product_names=[] 
brands=[] 
colors=[] 
sizes=[] 
fabrics=[] 
ids=[]

categorys=[] 

row = dict()
temp_list=[]
errors_links = []

for num, v in enumerate(dfV["link V"]):
    
    if num not in dead_page_list and num not in num_list and num>357:

        driverV.get(v) 
        time.sleep(5)
        soup = BeautifulSoup(driverV.page_source, 'html.parser')
        row = dict()
        row['num']=num

        brands.append("avenue")
        row['brand']= "avenue"

        name, price, idp= name_price_idV(soup)

        product_names.append(name)
        row['product_names']= name

        prices.append(price)
        row['prices']= price

        ids.append(idp)
        row['ids']= idp

        rating=ratingV(driverV)
        ratings.append(rating)
        row['ratings']= rating

        size=sizeV(soup)
        sizes.append(size)
        row['sizes']= size

        if (size==None):

            errors_links.append(num)
            print(num)


        fabric=fabricV(driverV)
        fabrics.append(fabric)
        row['fabrics']= fabric

        color=colorV(soup)
        colors.append(color)
        row['colors']= color

        temp_list.append(row)

        if num%1==0:
            df = pd.DataFrame(temp_list)
            df.to_csv("avenue_prodoct_info_temp_fix35.csv", index=False)


df

,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,358,avenue,Amalia Blouse - ink,39.5,Style # 217938INK,5.0,"[14, 16, 18, 20, 22-24, 26-28, 30-32]",Main Body: 100% Polyester.\nTrim: 100% Cotton.,Ink
1,361,avenue,Teresa Pleat Shirt - neutral,39.5,Style # 217942WILDBEAUTY,5.0,"[14, 16, 18, 20, 22-24, 26-28, 30-32]",100% Polyester,Wild Beauty
2,362,avenue,Color Block Polar Fleece Pullover - ivory,14.99,Style # 218006IVORYBLK,5.0,[None],100% Polyester.,Ivory/black
3,363,avenue,Knit Pleat Skirt - black,31.6,Style # 217886BLACK,4.0,[None],"Skirt: 63% Polyester, 33% Viscose, 4% Elastane.",Black
4,364,avenue,Harlow Frill Tunic - brown,31.6,Style # 218116PAISLEYMIXSTRIPE,3.5,[None],100% Polyester.,Paisley Mix Stripe
...,...,...,...,...,...,...,...,...,...
534,2621,avenue,High Rise Jegging Mid Wash - petite,9,Style # 207172PTMDWS,3.5,"[0X, 1X, 2X, 3X, 4X, 5X]","78% Cotton, 20% Polyester, 2% Elastane.",Mid Wash
535,2622,avenue,High Rise Jegging Dark Wash - petite,19.99,Style # 207172PTDKWS,3.5,"[0X, 1X, 2X, 3X, 4X, 5X]","78% Cotton, 20% Polyester, 2% Elastane.",Dark Wash
536,2631,avenue,Luxe Cool Hand Slimming Pant Gray - average,None,Style # 204374480GREYAVERAGE,4.5,"[14, 16, 18, 20, 22, 24, 26, 28, 30, 32]","72% Polyester, 24% Rayon, 4% Spandex. Fabric h...",Grey
537,2633,avenue,Wonderwall Coat - black,None,Style # 138764BLACK,5.0,"[12, 14, 16, 18, 20, 22, 24]","Main: 95% Polyester, 4% Viscose, 1% Nylon. Lin...",Black


combained final table:

In [25]:
df1= pd.read_csv("avenue_prodoct_info_temp_fix20.csv")
df2= pd.read_csv("avenue_prodoct_info_temp_fix25.csv")
df3= pd.read_csv("avenue_prodoct_info_temp_fix30.csv")
df4= pd.read_csv("avenue_prodoct_info_temp_fix35.csv")

df_final = df1.append([df2, df3,df4], ignore_index=True)

df_final

C:\Users\linoy\AppData\Local\Temp\ipykernel_41812\1263138541.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df1.append([df2, df3,df4], ignore_index=True)


,num,brand,product_names,prices,ids,ratings,sizes,fabrics,colors
0,0,avenue,Ashley Pintuck Top - terracotta,34.50,Style # 217814TERRACOTTA,0.0,[None],"65% Viscose, 35% Polyester.",Terracotta
1,1,avenue,Elouise Maxi Dress - orange,39.50,Style # 217671SANDSOFTIME,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Viscose.,Sands Of Time
2,2,avenue,Elouise Top - orange,34.50,Style # 217670SANDSOFTIME,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...",100% Viscose.,Sands Of Time
3,3,avenue,Button Knit Cardigan - cream,49.50,Style # 217660CREAM,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","52% Acrylic, 26% Polyester, 22% Nylon.",Cream
4,4,avenue,Button Knit Cardigan - camel,49.50,Style # 217660CAMEL,0.0,"['14', '16', '18', '20', '22-24', '26-28', '30...","52% Acrylic, 26% Polyester, 22% Nylon.",Camel
...,...,...,...,...,...,...,...,...,...
2621,2621,avenue,High Rise Jegging Mid Wash - petite,9.00,Style # 207172PTMDWS,3.5,"['0X', '1X', '2X', '3X', '4X', '5X']","78% Cotton, 20% Polyester, 2% Elastane.",Mid Wash
2622,2622,avenue,High Rise Jegging Dark Wash - petite,19.99,Style # 207172PTDKWS,3.5,"['0X', '1X', '2X', '3X', '4X', '5X']","78% Cotton, 20% Polyester, 2% Elastane.",Dark Wash
2623,2631,avenue,Luxe Cool Hand Slimming Pant Gray - average,NaN,Style # 204374480GREYAVERAGE,4.5,"['14', '16', '18', '20', '22', '24', '26', '28...","72% Polyester, 24% Rayon, 4% Spandex. Fabric h...",Grey
2624,2633,avenue,Wonderwall Coat - black,NaN,Style # 138764BLACK,5.0,"['12', '14', '16', '18', '20', '22', '24']","Main: 95% Polyester, 4% Viscose, 1% Nylon. Lin...",Black


In [28]:
df_final.to_csv("avenue_prodoct_info_final_fixed_table.csv", index=False)